In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.models import Sequential
from keras.layers import Dense,Flatten

In [3]:
from keras.constraints import min_max_norm
from keras.layers import Layer
import tensorflow as tf
import keras.backend as K
import keras


from pyopenset import ModuleNormalization,AddNewClass


In [4]:
print(keras.__version__)
print(tf.__version__)

2.4.3
2.2.0


In [5]:
Modelo=Sequential()
Modelo.add(Flatten(input_shape=(28,28,1)))
Modelo.add(Dense(60) )
Modelo.add(ModuleNormalization(s=20.0  ))
Modelo.add(Dense(10,kernel_constraint=min_max_norm(1,1),bias_constraint=min_max_norm(0,0),activation='softmax'))


In [6]:
Modelo.compile('adam','categorical_crossentropy')

In [7]:
Modelo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 60)                47100     
_________________________________________________________________
module_normalization (Module (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                610       
Total params: 47,710
Trainable params: 47,710
Non-trainable params: 0
_________________________________________________________________


In [8]:
#Modelo.save_weights('M1.h5')
Modelo.save('M1.h5')

In [10]:
from keras.models import load_model

How to load a model with our custom layer

In [11]:
Mod=load_model('M1.h5',custom_objects={'ModuleNormalization':ModuleNormalization  })


In [12]:
Mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 60)                47100     
_________________________________________________________________
module_normalization_1 (Modu (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                610       
Total params: 47,710
Trainable params: 47,710
Non-trainable params: 0
_________________________________________________________________


In [13]:
from keras.datasets import mnist

Defining and optimizing a model on only 8 of the classes

In [21]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [22]:
from keras.utils import to_categorical

In [23]:
import numpy as np

Filtro=np.where(y_train<8)[0]
FiltroTeste=np.where(y_test<8)[0]

In [24]:
Y_train=to_categorical(y_train[Filtro],8)
Y_test=to_categorical(y_test[FiltroTeste],8)

In [25]:
Modelo=Sequential()
Modelo.add(Flatten(input_shape=(28,28,1)))
Modelo.add(Dense(60) )
Modelo.add(ModuleNormalization(s=10.0  ))
Modelo.add(Dense(8,kernel_constraint=min_max_norm(1,1),bias_constraint=min_max_norm(0,0),activation='softmax'))


In [26]:
Modelo.compile('adam','categorical_crossentropy',metrics=['accuracy'])
Modelo.fit(x_train[Filtro],Y_train,epochs=1)

1507/1507 [==============================] - 1s 847us/step - loss: 0.2441 - accuracy: 0.9278


In [27]:
Modelo.evaluate(x_test[FiltroTeste],Y_test)

251/251 [==============================] - 0s 633us/step - loss: 0.1848 - accuracy: 0.9466


[0.18477389216423035, 0.9466134309768677]

Adding Classes 8 and 9 to the model and showing the summary so the user may see Modelo1 has 9 classes, while Modelo2 has 10

In [28]:
Modelo1=AddNewClass(Modelo, x_train[np.where(y_train==8)[0]] )

In [29]:
Modelo2=AddNewClass(Modelo1, x_train[np.where(y_train==9)[0]] )

In [34]:
Modelo1.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1_input (InputLayer) [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 60)                47100     
_________________________________________________________________
module_normalization_2 (Modu (None, 60)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 9)                 549       
Total params: 47,649
Trainable params: 47,649
Non-trainable params: 0
_________________________________________________________________


In [30]:
Modelo2.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1_input (InputLayer) [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 60)                47100     
_________________________________________________________________
module_normalization_2 (Modu (None, 60)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                610       
Total params: 47,710
Trainable params: 47,710
Non-trainable params: 0
_________________________________________________________________


Evaluating on the extended dataset

In [31]:
AvaliacaoFim=to_categorical(y_test,10)

In [32]:
Modelo2.compile('adam','categorical_crossentropy',metrics=['accuracy'])

Modelo2.evaluate(x_test,AvaliacaoFim)

313/313 [==============================] - 0s 618us/step - loss: 0.4398 - accuracy: 0.8543


[0.43978115916252136, 0.8543000221252441]

In [33]:
from sklearn.metrics import classification_report
Prev=Modelo2.predict(x_test)
print(classification_report(np.argmax(AvaliacaoFim,axis=1),np.argmax(Prev,axis=1)))

             precision    recall  f1-score   support

          0       0.95      0.97      0.96       980
          1       0.96      0.97      0.96      1135
          2       0.93      0.89      0.91      1032
          3       0.87      0.90      0.89      1010
          4       0.88      0.71      0.78       982
          5       0.86      0.79      0.82       892
          6       0.92      0.96      0.94       958
          7       0.92      0.78      0.84      1028
          8       0.77      0.67      0.71       974
          9       0.60      0.88      0.71      1009

avg / total       0.87      0.85      0.86     10000

